In [10]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from numpy.linalg import inv

# Global Variable

In [11]:
data = pd.read_excel('data 4.xlsx',delimiter=';')
data = pd.DataFrame(data, dtype='float64')
data.head(10)

,contrast-45,energy-45,homogenity-45,correlation-45,dissimilarity-45,ASM-45,contrast-135,energy-135,homogenity-135,correlation-135,dissimilarity-135,ASM-135,target
0,1.766642e+09,0.047762,0.290081,0.904779,201117057.0,0.003511,763683389.0,0.059255,0.404445,0.958808,201117057.0,0.003511,2.0
1,1.668317e+09,0.051764,0.291926,0.891612,199555184.0,0.004040,748656633.0,0.063563,0.403651,0.951372,199555184.0,0.004040,2.0
2,1.778061e+09,0.048937,0.288560,0.895871,205075808.0,0.003605,796003344.0,0.060044,0.398835,0.953375,205075808.0,0.003605,2.0
3,1.907568e+09,0.046138,0.279958,0.904327,209176143.0,0.003275,828993311.0,0.057227,0.393848,0.958502,209176143.0,0.003275,2.0
4,1.556532e+09,0.052248,0.307439,0.909354,189570792.0,0.004109,687430323.0,0.064104,0.420484,0.960026,189570792.0,0.004109,2.0
5,1.705768e+09,0.051676,0.288026,0.881365,201849498.0,0.004039,762620959.0,0.063551,0.400160,0.946930,201849498.0,0.004039,2.0
6,1.683875e+09,0.051220,0.298436,0.893320,202362319.0,0.003916,792157191.0,0.062579,0.405263,0.949808,202362319.0,0.003916,2.0
7,1.796451e+09,0.050646,0.285525,0.886239,202241918.0,0.003944,768489409.0,0.062799,0.401156,0.951334,202241918.0,0.003944,2.0
8,1.718631e+09,0.049792,0.293024,0.895410,200483835.0,0.003767,765063545.0,0.061376,0.405954,0.953453,200483835.0,0.003767,2.0
9,1.586000e+09,0.050746,0.302551,0.904782,191670011.0,0.003910,696421405.0,0.062531,0.415784,0.958163,191670011.0,0.003910,2.0


# Pembagian Data

In [12]:
df = data.drop(columns = ['target'])
minmax = preprocessing.MinMaxScaler()
np_scaled = minmax.fit_transform(df)
df = pd.DataFrame(np_scaled,columns=df.columns)

df_uji = df.iloc[:13, :]
df_uji = df_uji.append(df.iloc[54:, :])
df_latih = df.drop(index = df_uji.index)

df_uji_target = data.loc[:13, 'target']
df_uji_target = df_uji_target.append(data.loc[54:, 'target'])
df_latih_target = data['target'].drop(index = df_uji_target.index)

# Bobot

In [13]:
matriks_bobotInput = [[-0.4, 0.2, 0.1, -0.2, 0, 0.4, -0.3, 0.3, -0.1, 0.25, -0.45, 0.1]]
bobot = pd.DataFrame(matriks_bobotInput, columns = df_latih.columns)
bobot = bobot.transpose()
bobot.head(5)

,0
contrast-45,-0.4
energy-45,0.2
homogenity-45,0.1
correlation-45,-0.2
dissimilarity-45,0.0


# Training

In [14]:
def hitung_h():
    hinit = df_latih.dot(bobot)
    h = (1/(1+np.exp(-1*hinit)))
    return h
def transpose(df):
    return df.transpose()
def hitung_ht():
    trans = transpose(hitung_h())
    h = trans.dot(hitung_h())
    return h
def invers(df):
    inve = pd.DataFrame(inv(df))
    return inve
def hitung_hplus():
    tran = transpose(hitung_h())
    inve = invers(hitung_ht())
    return inve.dot(tran)
def beta_topi():
    hplus = hitung_hplus()
    beta = np.sum(hplus * df_latih_target,axis=1)
    return beta

# Testing

In [15]:
def hitung_huji():
    hinit = df_uji.dot(bobot)
    h = (1/(1+np.exp(-1*hinit)))
    return h
def peramalan():
    h = hitung_huji()
    beta = beta_topi()
    peramalan = np.sum(h * beta)
    return peramalan
def mape():
    beta = peramalan()
    mape = np.sum(np.absolute((beta-df_uji_target)/df_uji_target*100))/len(data)
    return mape
mape()

21.406505658898485